In [6]:
import pandas as pd 
df=pd.read_csv("data job posts.csv")


In [7]:
df = df.drop(['AnnouncementCode', 'Term', 'Eligibility', 'Audience', 'StartDate', 'Duration','RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach', 'Year', 'Month', 'IT'], axis=1)

In [8]:
import pandas as pd
import re

# Remove irrelevant information
# Clean the job title
df['Title'] = df['Title'].apply(lambda x: str(x))
df['Title'] = df['Title'].apply(lambda x: re.sub('[^a-z\s]', '', x))
df['Title'] = df['Title'].str.strip()


# Clean the JobRequirment
df['JobRequirment'] = df['JobRequirment'].apply(lambda x: str(x))
df['JobRequirment'] = df['JobRequirment'].apply(lambda x: re.sub('[^a-z\s]', '', x))
df['JobRequirment'] = df['JobRequirment'].str.strip()

# Extract the company name
df['Company'] = df['Company'].apply(lambda x: str(x).split(',')[0] if pd.notna(x) else '')
df['Company'] = df['Company'].str.strip()

# Extract the location
df['Location'] = df['Location'].apply(lambda x: x.split(',')[0] if pd.notna(x) else '')
df['Location'] = df['Location'].str.strip()

# Extract the required JobDescription
df['JobDescription'] = df['JobDescription'].apply(lambda x: str(x))
df['JobDescription'] = df['JobDescription'].apply(lambda x: re.findall('(?:requirement|qualification)[s]?[:]?[\\n\\s]*(.*?)(?:responsibilit|how to apply)', x))
df['JobDescription'] = df['JobDescription'].apply(lambda x: [item.strip() for item in x])
df['JobDescription'] = df['JobDescription'].apply(lambda x: ' '.join(x))




In [9]:
#model take a lot of time to run to we will take only 2000 rows
df = df.iloc[:2000]

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df))

# Prepare the training data
train_texts = list(df['JobDescription'])
train_labels = list(range(len(df)))
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

class JobPostingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JobPostingDataset(train_encodings, train_labels)

# Fine-tune the pre-trained BERT model on the training data
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained('/finetune/model')
tokenizer.save_pretrained('/finetune/tokenizer')

c:\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\21653\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predi

{'loss': 7.6521, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.08}


  5%|▌         | 20/375 [01:30<13:08,  2.22s/it]

{'loss': 7.6636, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.16}


  8%|▊         | 30/375 [01:50<11:31,  2.00s/it]

...

 99%|█████████▊| 370/375 [12:31<00:08,  1.63s/it]

{'loss': 7.6134, 'learning_rate': 3.7e-05, 'epoch': 2.96}


100%|██████████| 375/375 [12:40<00:00,  1.83s/it]

{'train_runtime': 768.5411, 'train_samples_per_second': 7.807, 'train_steps_per_second': 0.488, 'train_loss': 7.6430610656738285, 'epoch': 3.0}


100%|██████████| 375/375 [12:40<00:00,  2.03s/it]


In [ ]:
from transformers import pipeline

# load the fine-tuned model
model = pipeline('text2text-generation', model='/finetune/model', tokenizer='/finetune/tokenizer')

# define a function to perform semantic search
def search_jobs(query, location=None, max_results=10):
    # preprocess the query and location (if provided)
    query = query.lower()
    if location is not None:
        location = location.lower()
    
    # retrieve all job postings that match the location (if provided)
    if location is not None:
        matching_jobs = df[df['Location'].str.lower().str.contains(location)]
    else:
        matching_jobs = df
    
    # perform semantic search to retrieve the most relevant job postings
    results = []
    for i, row in matching_jobs.iterrows():
        job_description = row['JobDescription']
        score = model(query, job_description)[0]['score']
        results.append((i, score))
    
    # sort the results by relevance score and return the top results
    results = sorted(results, key=lambda x: x[1], reverse=True)[:max_results]
    return [matching_jobs.loc[i] for i, _ in results]


: 